<a href="https://colab.research.google.com/github/danykoud/Test-/blob/main/NLP_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from collections import defaultdict
import math
import numpy as np
import string

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Do not change anything in this cell

# Punctuation characters
punct = set(string.punctuation)

# Morphology rules used to assign unknown word tokens
noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]


def get_word_tag(line, vocab): 
    if not line.split():
        word = "--n--"
        tag = "--s--"
        return word, tag
    else:
        word, tag = line.split()
        if word not in vocab: 
            # Handle unknown words
            word = assign_unk(word)
        return word, tag
    return None 


def preprocess(vocab, data_fp):
    """
    Preprocess data
    """
    orig = []
    prep = []

    # Read data
    with open(data_fp, "r") as data_file:

        for cnt, word in enumerate(data_file):

            # End of sentence
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Handle unknown words
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep


def assign_unk(tok):
    """
    Assign unknown word tokens
    """
    # Digits
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Punctuation
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Upper-case
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Nouns
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Verbs
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Adjectives
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Adverbs
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"

    return "--unk--"


**Introduction to Dataset**

This assignment will use two tagged data sets. One data set (train.pos) will be used for training. The other (test.pos) for testing. The tagged training data has been preprocessed to form a vocabulary (hmv.txt). The words in the vocabulary are words from the training set that were used two or more times. The vocabulary is augmented with a set of 'unknown word tokens'.

The training set will be used to create the emission, transmission and tag counts.

The test set (test.pos) is read in to create y. This contains both the test text and the true tag. The test set has also been preprocessed to remove the tags to form test_words.txt.

A POS tagger will necessarily encounter words that are not in its datasets. A set of unknown-tokens, such as '--unk-verb--' or '--unk-noun--' will replace the unknown words in both the training and test corpus and will appear in the emission, transmission and tag data structures.

**Task 1** - Training the model: (20 points)
Write a program that takes in the training_corpus and returns the three dictionaries mentioned above transition_counts, emission_counts, and tag_counts.

**emission_counts:** maps (tag, word) to the number of times it happened.
transition_counts: maps (prev_tag, tag) to the number of times it has appeared.
tag_counts: maps (tag) to the number of times it has occured.
Tip: Make use of defaultdict as a standard Python dictionary throws a KeyError if you try to access an item with a key that is not currently in the dictionary. In contrast, the defauweltdict will create an item of the type of the argument, in this case an integer with the default value of 0.

In [ ]:
path1 = "/content/drive/MyDrive/files/train.pos"
path2= "/content/drive/MyDrive/files/test.pos"
path3= "/content/drive/MyDrive/files/hmv.txt"
# Do not change anything in this cell
# load in the training corpus
with open(path1, 'r') as f:
    training_corpus = f.readlines()

# read the vocabulary data, split by each line of text, and save the list
with open(path3, 'r') as f:
    voc_l = f.read().split('\n')

# vocab: dictionary that has the index of the corresponding words
vocab = {} 

# Get the index of the corresponding words. 
for i, word in enumerate(sorted(voc_l)): 
    vocab[word] = i       
    

cnt = 0
for k,v in vocab.items():
    cnt += 1
    if cnt > 20:
        break

# load in the test corpus
with open(path2, 'r') as f:
    y = f.readlines()

#corpus without tags, preprocessed
_, prep = preprocess(vocab,path2)

In [ ]:
 def create_dictionaries(training_corpus: list, vocab: dict):
     """Creat the three training dictionaries

     Args: 
        ``training_corpus``: a corpus where each line has a word followed by its tag.
        ``vocab``: a dictionary where keys are words in vocabulary and value is an index
     Returns: 
        ``emission_counts``: a dictionary where the keys are (tag, word) and the values are the counts
        ``transition_counts``: a dictionary where the keys are (prev_tag, tag) and the values are the counts
        ``tag_counts``: a dictionary where the keys are the tags and the values are the counts
     """

     # initialize the dictionaries 
     emission_counts = defaultdict(int)
     transition_counts = defaultdict(int)
     tag_counts = defaultdict(int)

     # Initialize "prev_tag" (previous tag) with the start state, denoted by '--s--'
     prev_tag = '--s--' 

     
     i = 0 
     for word_tag in training_corpus:
         i += 1

         if i % 50000 == 0:
             print(f"word count = {i}")
         # get the word and tag using the get_word_tag  function
         word, tag = get_word_tag(word_tag, vocab)

        #  increment the transition and emission count by one 
         transition_counts[(prev_tag, tag)] += 1
         emission_counts[(tag, word)] += 1

         # Increment the tag count
         tag_counts[tag] += 1
         prev_tag = tag

         ### END CODE HERE ###

     return emission_counts, transition_counts, tag_counts


In [ ]:
# Do not change anything in this cell

emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)

word count = 50000
word count = 100000
word count = 150000
word count = 200000
word count = 250000
word count = 300000
word count = 350000
word count = 400000
word count = 450000
word count = 500000
word count = 550000
word count = 600000
word count = 650000
word count = 700000
word count = 750000
word count = 800000
word count = 850000
word count = 900000
word count = 950000
Number of POS tags (number of 'states'): 46
View these POS tags (states)
['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


**Task 2:** Testing the model: (10 points)

Implement predict_pos that computes the accuracy of your model.

To assign a part of speech to a word, assign the most frequent POS for that word in the training set.

Then evaluate how well this approach works. 
Each time you predict based on the most frequent POS for the given word, check whether the actual POS of that word is the same. If so, the prediction was correct!

Calculate the accuracy as the number of correct predictions divided by the total number of words for which you predicted the POS tag.

In [ ]:
def predict_pos(prep, y, emission_counts, vocab, states):
    '''
    Input: 
        prep: a preprocessed version of 'y'. A list with the 'word' component of the tuples.
        y: a corpus composed of a list of tuples where each tuple consists of (word, POS)
        emission_counts: a dictionary where the keys are (tag,word) tuples and the value is the count
        vocab: a dictionary where keys are words in vocabulary and value is an index
        states: a sorted list of all possible tags for this assignment
    Output: 
        accuracy: Number of times you classified a word correctly
    '''
    
    # Initialize the number of correct predictions 
    num_correct_pred = 0
    
    # Get the (tag, word) tuples, stored as a set
    words = set(emission_counts.keys())  
    
    # Get the number of (word, POS) tuples in the corpus 'y'
    total_num_words = len(y)
    for word, y_tup in zip(prep, y): 

        # Split the (word, POS) string into a list of two items
        y_tup_split = y_tup.split()
        
        # Verify that y_tup contain both word and POS
        if len(y_tup_split) == 2:
            
            # Set the true POS label for this word
            true_pos_label = y_tup_split[1]

        else:
            # If the y_tup didn't contain word and POS, go to next word
            continue
    
        count_final = 0
        pos_final = ''
        
        # If the word is in the vocabulary...
        if word in vocab:
            for pos in states:

            ### START CODE HERE (Replace instances of 'None' with your code) ###
                        
                # define the key as the tuple containing the POS and word
                key = (pos, word)

                # check if the (pos, word) key exists in the emission_counts dictionary
                if key in emission_counts: # complete this line

                # get the emission count of the (pos,word) tuple 
                    count = emission_counts[key]

                    # keep track of the POS with the largest count
                    if count > count_final: # complete this line

                        # update the final count (largest count)
                        count_final = count

                        # update the final POS
                        pos_final = pos

            # If the final POS (with the largest count) matches the true POS:
            if pos_final == true_pos_label : 
                
                # Update the number of correct predictions
                num_correct_pred += 1
            
    ### END CODE HERE ###
    accuracy = num_correct_pred  / total_num_words
    
    return accuracy

In [ ]:
# Do not change anything in this cell
accuracy_predict_pos = predict_pos(prep, y, emission_counts, vocab, states)
print(f"Accuracy of prediction using predict_pos is {accuracy_predict_pos:.4f}")

Accuracy of prediction using predict_pos is 0.1077


**Task 3 **

**- Hidden Markov Model**: (20 points)

Hidden Markov Model: The HMM is one of the most commonly used algorithms in Natural Language Processing. The Markov Model contains a number of states and the probability of transition between those states. In this case, the states are the parts-of-speech. A Markov Model utilizes a transition matrix, A. A Hidden Markov Model adds an observation or emission matrix B which describes the probability of a visible observation when we are in a particular state. In this case, the emissions are the words in the corpus. The state, which is hidden, is the POS tag of that word.

The smoothing was done as follows:

𝑃(𝑡𝑖|𝑡𝑖−1)=𝐶(𝑡𝑖−1,𝑡𝑖)+𝛼𝐶(𝑡𝑖−1)+𝛼∗𝑁(3)

𝑁  is the total number of tags
𝐶(𝑡𝑖−1,𝑡𝑖)  is the count of the tuple (previous POS, current POS) in transition_counts dictionary.

𝐶(𝑡𝑖−1)  is the count of the previous POS in the tag_counts dictionary.
𝛼  is a smoothing parameter.

Implement the create_transition_matrix below for all tags.

In [ ]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    ''' 
    Input: 
        alpha: number used for smoothing
        tag_counts: a dictionary mapping each tag to its respective count
        transition_counts: transition count for the previous word and tag
    Output:
        A: matrix of dimension (num_tags,num_tags)
    '''
    # sorted list of all tags
    all_tags = sorted(tag_counts.keys())
    
    # length of the list of all tags
    len_tags = len(all_tags)
    
    # Initialize the transition matrix 'A'
    A = np.zeros((len_tags,len_tags))
    
    # Get  previous POS, current POS from  transition tuples 
    uni_trans_tup = set(transition_counts.keys())
    
    
   
    for x in range(len_tags):
        for y in range(len_tags):
            count = 0
            key = (all_tags[x],all_tags[y])
            if key in uni_trans_tup: 
                count = transition_counts[key]

            count_prev_tag = tag_counts[all_tags[x]]
            
            A[x,y] = (count + alpha ) / ( count_prev_tag + alpha * len_tags)

    ### END CODE HERE ###
    
    return A

In [ ]:
# Do not change anything in this cell
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)
# Testing your function
print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[3,1]:.4f}")

print("View a subset of transition matrix A")
A_sub = pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )
print(A_sub)

A at row 0, col 0: 0.000007040
A at row 3, col 1: 0.1691
View a subset of transition matrix A
              RBS            RP           SYM        TO            UH
RBS  2.217069e-06  2.217069e-06  2.217069e-06  0.008870  2.217069e-06
RP   3.756509e-07  7.516775e-04  3.756509e-07  0.051089  3.756509e-07
SYM  1.722772e-05  1.722772e-05  1.722772e-05  0.000017  1.722772e-05
TO   4.477336e-05  4.472863e-08  4.472863e-08  0.000090  4.477336e-05
UH   1.030439e-05  1.030439e-05  1.030439e-05  0.061837  3.092348e-02


Now you will create the B transition matrix which computes the emission probability.

You will use smoothing as defined below:

𝑃(𝑤𝑖|𝑡𝑖)=𝐶(𝑡𝑖,𝑤𝑜𝑟𝑑𝑖)+𝛼𝐶(𝑡𝑖)+𝛼∗𝑁(4)

- 𝐶(𝑡𝑖,𝑤𝑜𝑟𝑑𝑖)  is the number of times  𝑤𝑜𝑟𝑑𝑖  was associated with  𝑡𝑎𝑔𝑖  in the training data (stored in emission_counts dictionary).

- 𝐶(𝑡𝑖)  is the number of times  𝑡𝑎𝑔𝑖  was in the training data (stored in tag_counts dictionary).

- 𝑁  is the number of words in the vocabulary

- 𝛼  is a smoothing parameter.

The matrix B is of dimension (num_tags, N), where num_tags is the number of possible parts-of-speech tags.

Implement the create_emission_matrix below that computes the B emission probabilities matrix. Your function takes in  𝛼 , the smoothing parameter, tag_counts, which is a dictionary mapping each tag to its respective count, the emission_counts dictionary where the keys are (tag, word) and the values are the counts.

In [ ]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    '''
    Input: 
        alpha: tuning parameter used in smoothing 
        tag_counts: a dictionary mapping each tag to its respective count
        emission_counts: a dictionary where the keys are (tag, word) and the values are the counts
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output:
        B: a matrix of dimension (num_tags, len(vocab))
    '''
    
    # get the number of POS tag
    num_tags = len(tag_counts)
    
    # Get a list of all POS tags
    all_tags = sorted(tag_counts.keys())
    
    # Get the total number of unique words in the vocabulary
    num_words = len(vocab)
    
    # Initialize the emission matrix B with places for
    # tags in the rows and words in the columns
    B = np.zeros((num_tags, num_words))
    # from the keys of the emission_counts dictionary
    emission_keys = set(list(emission_counts.keys()))
       
   
    for i in range(num_tags): 
        for j, word in enumerate(vocab): 
            count = 0
            key =  (all_tags[i], word)
            
            if key in emission_keys:        
                count = emission_counts[key] # Get the count of (POS tag, word) from the emission_counts d
                
            # Get the count of the POS tag
            count_tag = tag_counts[all_tags[i]]
                
            # Apply smoothing and store the smoothed value into the emission matrix B 
            B[i,j] = (count + alpha) / (count_tag + alpha * num_words)

    return B

In [ ]:
# Do not change anything in this cell
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))
print(f"View Matrix position at row 0, column 0: {B[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {B[3,1]:.9f}")
cidx  = ['725','adroitly','engineers', 'promoted', 'synergy']
cols = [vocab[a] for a in cidx]
rvals =['CD','NN','NNS', 'VB','RB','RP']
rows = [states.index(a) for a in rvals]
B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns = cidx )
print(B_sub)

View Matrix position at row 0, column 0: 0.000006032
View Matrix position at row 3, column 1: 0.000000720
              725      adroitly     engineers      promoted       synergy
CD   8.201296e-05  2.732854e-08  2.732854e-08  2.732854e-08  2.732854e-08
NN   7.521128e-09  7.521128e-09  7.521128e-09  7.521128e-09  2.257091e-05
NNS  1.670013e-08  1.670013e-08  4.676203e-04  1.670013e-08  1.670013e-08
VB   3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08
RB   3.226454e-08  6.456135e-05  3.226454e-08  3.226454e-08  3.226454e-08
RP   3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07


# Task 4 

- **Viterbi Algorithm**: (50 points)

In this part of the assignment you will implement the Viterbi algorithm which makes use of dynamic programming. Specifically, you will use your two matrices, A and B to compute the Viterbi algorithm. We have decomposed this process into three main steps for you.

Initialization - In this part you initialize the best_paths and best_probabilities matrices that you will be populating in feed_forward.
Feed forward - At each step, you calculate the probability of each path happening and the best paths up to that point.
Feed backward: This allows you to find the best path with the highest probabilities.
You will start by initializing two matrices of the same dimension.

best_probs: Each cell contains the probability of going from one POS tag to a word in the corpus.

best_paths: A matrix that helps you trace through the best possible path in the corpus.

Write a program below that initializes the best_probs and the best_paths matrix.

Both matrices will be initialized to zero except for column zero of best_probs.

Column zero of best_probs is initialized with the assumption that the first word of the corpus was preceded by a start token ("--s--").
This allows you to reference the A matrix for the transition probability
Here is how to initialize column 0 of best_probs:

The probability of the best path going from the start index to a given POS tag indexed by integer  𝑖  is denoted by  best_probs[𝑠𝑖𝑑𝑥,𝑖] .

This is estimated as the probability that the start tag transitions to the POS denoted by index  𝑖 :  𝐀[𝑠𝑖𝑑𝑥,𝑖]  AND that the POS tag denoted by  𝑖  emits the first word of the given corpus, which is  𝐁[𝑖,𝑣𝑜𝑐𝑎𝑏[𝑐𝑜𝑟𝑝𝑢𝑠[0]]] .

Conceptually, it looks like this:  best_probs[𝑠𝑖𝑑𝑥,𝑖]=𝐀[𝑠𝑖𝑑𝑥,𝑖]×𝐁[𝑖,𝑐𝑜𝑟𝑝𝑢𝑠[0]] 

In order to avoid multiplying and storing small values on the computer, we'll take the log of the product, which becomes the sum of two logs:

𝑏𝑒𝑠𝑡_𝑝𝑟𝑜𝑏𝑠[𝑖,0]=𝑙𝑜𝑔(𝐴[𝑠𝑖𝑑𝑥,𝑖])+𝑙𝑜𝑔(𝐵[𝑖,𝑣𝑜𝑐𝑎𝑏[𝑐𝑜𝑟𝑝𝑢𝑠[0]] 
Also, to avoid taking the log of 0 (which is defined as negative infinity), the code itself will just set  𝑏𝑒𝑠𝑡_𝑝𝑟𝑜𝑏𝑠[𝑖,0]=𝑓𝑙𝑜𝑎𝑡(′−𝑖𝑛𝑓′)  when  𝐴[𝑠𝑖𝑑𝑥,𝑖]==0 

So the implementation to initialize  𝑏𝑒𝑠𝑡_𝑝𝑟𝑜𝑏𝑠  looks like this:

𝑖𝑓𝐴[𝑠𝑖𝑑𝑥,𝑖]<>0:𝑏𝑒𝑠𝑡_𝑝𝑟𝑜𝑏𝑠[𝑖,0]=𝑙𝑜𝑔(𝐴[𝑠𝑖𝑑𝑥,𝑖])+𝑙𝑜𝑔(𝐵[𝑖,𝑣𝑜𝑐𝑎𝑏[𝑐𝑜𝑟𝑝𝑢𝑠[0]]]) 

𝑖𝑓𝐴[𝑠𝑖𝑑𝑥,𝑖]==0:𝑏𝑒𝑠𝑡_𝑝𝑟𝑜𝑏𝑠[𝑖,0]=𝑓𝑙𝑜𝑎𝑡(′−𝑖𝑛𝑓′)

In [ ]:
def initialize(states, tag_counts, A, B, corpus, vocab):
    '''
    Input: 
        states: a list of all possible parts-of-speech
        tag_counts: a dictionary mapping each tag to its respective count
        A: Transition Matrix of dimension (num_tags, num_tags)
        B: Emission Matrix of dimension (num_tags, len(vocab))
        corpus: a sequence of words whose POS is to be identified in a list 
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output:
        best_probs: matrix of dimension (num_tags, len(corpus)) of floats
        best_paths: matrix of dimension (num_tags, len(corpus)) of integers
    '''
    # Get the total number of unique POS tags
    num_tags = len(tag_counts)
    
    # Initialize best_probs matrix 
    best_probs = np.zeros((num_tags, len(corpus)))
    
    # Initialize best_paths matrix
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)
    
    # Define the start token
    s_idx = states.index("--s--")
    for i in range(num_tags) : # complete this line
        
        # Handle the special case when the transition from start token to POS tag i is zero
        if A[0,i] == 0:      
            best_probs[i,0] = float("-inf")
        else: 
            # Initialize best_probs at POS tag 'i', column 0
            best_probs[i,0] = math.log(A[s_idx,i])  +  math.log(B[i,vocab[corpus[0]]])
    return best_probs, best_paths

In [ ]:
# Do not change anything in this cell

best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)
print(f"best_probs[0,0]: {best_probs[0,0]:.4f}") 
print(f"best_paths[2,3]: {best_paths[2,3]:.4f}")

best_probs[0,0]: -22.6098
best_paths[2,3]: 0.0000


Implement the viterbi_forward algorithm and store the best_path and best_prob for every possible tag for each word in the matrices best_probs and best_tags using the pseudo code below.

`for each word in the corpus

for each POS tag type that this word may be

    for POS tag type that the previous word could be

        compute the probability that the previous word had a given POS tag, that the current word has a given POS tag, and that the POS tag would emit this current word.

        retain the highest probability computed for the current word

        set best_probs to this highest probability

        set best_paths to the index 'k', representing the POS tag of the previous word which produced the highest probability `


In [ ]:
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab):
    '''
    Input: 
        A, B: The transiton and emission matrices respectively
        test_corpus: a list containing a preprocessed corpus
        best_probs: an initilized matrix of dimension (num_tags, len(corpus))
        best_paths: an initilized matrix of dimension (num_tags, len(corpus))
        vocab: a dictionary where keys are words in vocabulary and value is an index 
    Output: 
        best_probs: a completed matrix of dimension (num_tags, len(corpus))
        best_paths: a completed matrix of dimension (num_tags, len(corpus))
    '''
    # Get the number of unique POS tags (which is the num of rows in best_probs)
    num_tags = best_probs.shape[0]
    for i in range(1, len(test_corpus)): 
        
        # Print number of words processed, every 5000 words
        if i % 5000 == 0:
            print("Words processed: {:>8}".format(i))
        for j in range(num_tags): 
            # Initialize best_prob for word i to negative infinity
            best_prob_i = float("-inf")
            # Initialize best_path for current word i to None
            best_path_i = None

            # For each POS tag that the previous word can be:
            for k in range(num_tags): # complete this line
            
                # Calculate the probability = 
                # best probs of POS tag k, previous word i-1 + 
                # log(prob of transition from POS k to POS j) + 
                # log(prob that emission of POS j is word i)
                prob = best_probs[k, i-1] + math.log(A[k,j]) + math.log(B[j, vocab[test_corpus[i]]])

                # check if this path's probability is greater than
                # the best probability up to and before this point
                if prob > best_prob_i:     
                    # Keep track of the best probability
                    best_prob_i = prob
                    
                    # keep track of the POS tag of the previous word
                    best_path_i = k

            # Save the best probability for the 
            # given current word's POS tag
            best_probs[j,i] = best_prob_i
            
            # Save the unique integer ID of the previous POS tag
            best_paths[j,i] = best_path_i

    return best_probs, best_paths

In [ ]:
# Do not change anything in this cell
# this will take a few minutes to run => processes ~ 30,000 words
best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)

Words processed:     5000
Words processed:    10000
Words processed:    15000
Words processed:    20000
Words processed:    25000
Words processed:    30000


Implement the viterbi_backward algorithm, which returns a list of predicted POS tags for each word in the corpus.

Note that the numbering of the index positions starts at 0 and not 1.

m is the number of words in the corpus.
So the indexing into the corpus goes from 0 to m - 1.

Also, the columns in best_probs and best_paths are indexed from 0 to m - 1

**In Step 1:**
Loop through all the rows (POS tags) in the last entry of best_probs and find the row (POS tag) with the maximum value. Convert the unique integer ID to a tag (a string representation) using the dictionary states.

Referring to the three-word corpus described above:

z[2] = 28: For the word 'upward' at position 2 in the corpus, the POS tag ID is 28. Store 28 in z at position 2.
states(28) is 'RB': The POS tag ID 28 refers to the POS tag 'RB'.

pred[2] = 'RB': In array pred, store the POS tag for the word 'upward'.
In Step 2:

Starting at the last column of best_paths, use best_probs to find the most likely POS tag for the last word in the corpus.
Then use best_paths to find the most likely POS tag for the previous word.

Update the POS tag for each word in z and in preds.
Referring to the three-word example from above, read best_paths at column 2 and fill in z at position 1.
z[1] = best_paths[z[2],2]

The small test following the routine prints the last few words of the corpus and their states to aid in debug.

In [ ]:
def viterbi_backward(best_probs, best_paths, corpus, states):
    '''
    This function returns the best path.
    
    '''
    # Get the number of words in the corpus
    B= best_paths.shape[1] 
    # Initialize array arr, same length as the corpus
    arr = [None] * B
    
    # Get the number of unique POS tags
    num_tags = best_probs.shape[0]
    
    # Initialize the best probability for the last word
    best_prob_for_last_word = float('-inf')
    
    # Initialize pred array, same length as corpus
    pred = [None] * B
    for i in range(num_tags): # complete this line

        # If the probability of POS tag at row i
        # is better than the previosly best probability for the last word
        if best_probs[i,-1] > best_prob_for_last_word: 
            # Store the new best probability for the last word
            best_prob_for_last_word = best_probs[i,-1]
            arr[B - 1] = i
            
    # Convert the last word's predicted POS tag
    pred[B - 1] = states[i]
    #  Find the best POS tags by walking backward through the best_paths
    for j in range(B-1, 0, -1): 
        # Retrieve the unique integer ID of
        # the POS tag for the word at position 'j' in the corpus
        pos_tag_for_word_j = best_paths[arr[j], j]
#         print(arr[j])
        
        # retrieve the predicted POS for the word at position j-1 in the corpus
        arr[j - 1] = pos_tag_for_word_j
        # Get the previous word's POS tag in string form
        pred[j - 1] = states[pos_tag_for_word_j]     

    return pred

In [ ]:
# Do not change anything in cell

pred = viterbi_backward(best_probs, best_paths, prep, states)
m=len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

print('The third word is:', prep[3])
print('Your prediction is:', pred[3])
print('Your corresponding label y is: ', y[3])

The prediction for pred[-7:m-1] is: 
 ['--unk_upper--', '--unk_upper--', '--unk_upper--', '--unk_upper--', '--unk_upper--', '--unk_punct--'] 
 ['NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP'] 

The prediction for pred[0:8] is: 
 ['NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP'] 
 ['--unk_upper--', '--unk_upper--', '--unk_punct--', '--unk_upper--', '--unk_upper--', '--unk_upper--', '--unk_upper--']
The third word is: --unk_upper--
Your prediction is: NNP
Your corresponding label y is:  temperature	NN



In [ ]:
def compute_accuracy(pred, y):
    '''
    Input: 
        pred: a list of the predicted parts-of-speech 
        y: a list of lines where each word is separated by a '\t' (i.e. word \t tag)
    Output: 
        
    '''
    num = 0 #number of times that the prediction nd label match
    T = 0 #total number of examples that have valid labels
    
    # Zip together the prediction and the labels
    for prediction, y in zip(pred, y):
      
        # Split the label into the word and the POS tag
        word_postag = y.strip().split("\t")

        if len(word_postag) < 2: 
            continue 
        #  separately store  word and tag
        word, tag = [item.strip() for item in word_postag]      
#         print(tag, prediction) 
        if tag == prediction:
            num+= 1.0
        T += 1.0 
    return num/T

In [ ]:
# Do not change anything in cell
print(f"Accuracy of the Viterbi algorithm is {compute_accuracy(pred, y):.4f}")

Accuracy of the Viterbi algorithm is 0.1071
